## News Reporter Agentic System Using Semantic Kernel SDK and Azure AI Agent Service

In [ ]:
%pip install semantic-kernel

In [ ]:
%pip install azure-ai-projects azure-identity

#### Importing important libraries and utilities

In [ ]:
from semantic_kernel import Kernel
import os
import asyncio
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from dotenv import load_dotenv
from typing import Annotated
from semantic_kernel.functions.kernel_function_decorator import kernel_function
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import BingGroundingTool
load_dotenv()



In [ ]:
azure_ai_foundry_endpoint = os.getenv("AZURE_AI_FOUNDRY_ENDPOINT_PORTAL")
azure_openai_key = os.getenv("AZURE_AI_FOUNDRY_OPENAI_KEY")
azure_openai_deployment_name = os.getenv("AZURE_AI_FOUNDRY_OPENAI_DEPLOYMENT")
azure_openai_endpoint = os.getenv("AZURE_AI_FOUNDRY_OPENAI_ENDPOINT")
azure_openai_api_version = os.getenv("AZURE_AI_FOUNDRY_OPENAI_API_VERSION")
bing_connection_name = os.getenv("BING_CONNECTION_NAME")

In [73]:
project_client = AIProjectClient(
    endpoint=azure_ai_foundry_endpoint,
    credential=DefaultAzureCredential()
)

In [77]:
with AIProjectClient(
    endpoint=azure_ai_foundry_endpoint,
    credential=DefaultAzureCredential()
) as client:
    bing_connection = client.connections.get(
        name=bing_connection_name
    )
    conn_id = bing_connection.id  # Get the connection ID from the Bing connection

    # Initialize the Bing Grounding tool
    bing = BingGroundingTool(connection_id=conn_id)

    agent = client.agents.create_agent(
        model=azure_openai_deployment_name,  # Model deployment name
        name="bing-ai-assistant",  # Name of the agent
        instructions="You are a helpful agent",  # Instructions for the agent
        tools=bing.definitions,  # Attach the Bing Grounding tool
    )
    print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_oUfc1paPsPEkqaEF8AYKAdLH


In [78]:
# Create a thread for communication
thread = project_client.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

# Add a message to the thread
message = project_client.agents.messages.create(
    thread_id=thread.id,
    role="user",  # Role of the message sender
    content="What is the weather in Seattle today?",  # Message content
)
print(f"Created message, ID: {message['id']}")

Created thread, ID: thread_9oPwADQVtiBu0BfTKW9X2bs6
Created message, ID: msg_JKObgovpTHxkgcPy4KMHbd4E
Created message, ID: msg_JKObgovpTHxkgcPy4KMHbd4E


In [79]:
# Create and process an agent run
run = project_client.agents.runs.create_and_process(
    thread_id=thread.id,
    agent_id=agent.id,
    # tool_choice={"type": "bing_grounding"}  # optional, you can force the model to use Grounding with Bing Search tool
)
print(f"Run finished with status: {run.status}")

# Check if the run failed
if run.status == "failed":
    print(f"Run failed: {run.last_error}")

# Fetch and log all messages
messages = project_client.agents.messages.list(thread_id=thread.id)
for message in messages:
    print(f"Role: {message.role}, Content: {message.content}")

Run finished with status: RunStatus.COMPLETED
Role: MessageRole.AGENT, Content: [{'type': 'text', 'text': {'value': "Today's weather in Seattle, WA, includes a high of approximately 82°F (28°C) and a low of about 64°F (18°C), with generally clear skies【3:2†source】.", 'annotations': [{'type': 'url_citation', 'text': '【3:2†source】', 'start_index': 135, 'end_index': 147, 'url_citation': {'url': 'https://www.accuweather.com/en/us/seattle/98104/august-weather/351409', 'title': 'Seattle, WA Monthly Weather | AccuWeather'}}]}}]
Role: MessageRole.USER, Content: [{'type': 'text', 'text': {'value': 'What is the weather in Seattle today?', 'annotations': []}}]
Role: MessageRole.AGENT, Content: [{'type': 'text', 'text': {'value': "Today's weather in Seattle, WA, includes a high of approximately 82°F (28°C) and a low of about 64°F (18°C), with generally clear skies【3:2†source】.", 'annotations': [{'type': 'url_citation', 'text': '【3:2†source】', 'start_index': 135, 'end_index': 147, 'url_citation': {

#### Creating an "Agent" Plugin Class which will include native plugins to be fed into the Kernel

In [94]:
class Agents:
    @kernel_function(
        description="This function will be used to use an azure ai agent with web grounding capability using Bing Search API",
        name="WebSearchAgent"
    )
    def web_search_agent(
        self,
        query: Annotated[str, "The user query for which the contextual information needs to be fetched from the web"]
        
    ) -> Annotated[str, "The response from the web search agent"]:
        bing_connection = project_client.connections.get(name=bing_connection_name)
        conn_id = bing_connection.id
        bing = BingGroundingTool(connection_id=conn_id)
        
        agent = project_client.agents.create_agent(
            model=azure_openai_deployment_name,
            name="bing-assistant",
            instructions="You are a helpful assistant",
            tools=bing.definitions
        )
        thread = project_client.agents.threads.create()
    

        message = project_client.agents.messages.create(
            thread_id=thread.id,
            role="user",
            content=query,
        )

        run = project_client.agents.runs.create_and_process(
            thread_id=thread.id,
            agent_id=agent.id,
        )

        messages = project_client.agents.messages.list(thread_id=thread.id)
        
        for message in messages:
            print(f"Role: {message.role}, Content: {message.content}")
        # Return the content of the first message as the response
        return messages[0].content if messages else ""
    
    
    @kernel_function(
       description="This function will use an azure ai agent to prepare a script for a news reporter based on latest information for a specific topic",
         name="NewsReporterAgent"
   )
    def news_reporter_agent(
        self,
        topic: Annotated[str, "The topic for which the latest information/news has been fetched"],
        latest_news: Annotated[str,"The latest information for a specific topic"]
    ) -> Annotated[str, "the response from the NewsReporterAgent which is the script for a news reporter"]:

        agent = project_client.agents.create_agent(
        model=azure_openai_deployment_name,
        name="news-reporter",
        instructions="""You are a helpful assistant that is meant to prepare a script for a news reporter based on the latest information for a specific topic both of which you will be given.
            The news channel is named MSinghTV and the news reporter is named John. You will be given the topic and the latest information for that topic. Prepare a script for the news reporter John based on the latest information for the topic.""",
        )
        thread = project_client.agents.threads.create()
            
        message = project_client.agents.messages.create(
                thread_id=thread.id,
                role="user",
                content=f"""The topic is {topic} and the latest information is {latest_news}""",
            )
            
        run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
     
        messages = project_client.agents.messages.list(thread_id=thread.id)
        
        print("Script for the news reporter:")
        print("\n")    
        for message in messages:
            print(f"Role: {message.role}, Content: {message.content}")
        # Return the content of the first message as the response
        return messages[0].content if messages else ""
        

#### Creating the Kernel of the Semantic Kernel SDK

In [113]:
kernel = Kernel()

chat_completion = AzureChatCompletion(
        api_key=azure_openai_key,
        deployment_name=azure_openai_deployment_name,
        endpoint=azure_openai_endpoint,
        api_version=azure_openai_api_version
    )

kernel.add_service(
    chat_completion
)

#### Adding the Agentic Plugins as native plugins to our Kernel so created 

In [ ]:
agents_plugin = kernel.add_plugin(Agents(), "Agents")

In [115]:
from semantic_kernel.connectors.ai.open_ai import AzureChatPromptExecutionSettings 
from semantic_kernel.connectors.ai import FunctionChoiceBehavior
from semantic_kernel.contents import ChatHistory
from semantic_kernel.functions import KernelArguments
# Enable planning
execution_settings = AzureChatPromptExecutionSettings()
execution_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

In [116]:
 # Create a history of the conversation
history = ChatHistory()

In [117]:
userInput = None
while True:
        # Collect user input
        userInput = input("User > ")

        # Terminate the loop if the user says "exit"
        if userInput == "exit":
            break

        # Add user input to the history
        history.add_user_message(userInput)

        # 3. Get the response from the AI with automatic function calling
        result = await chat_completion.get_chat_message_content(
            chat_history=history,
            settings=execution_settings,
            kernel=kernel,
        )

        # Print the results
        print("Assistant > " + str(result))

        # Add the message from the agent to the chat history
        history.add_message(result)


Assistant > Could you let me know the topic or type of news you want me to prepare the script for John?


Function failed. Error: HTTP transport has already been closed. You may check if you're calling a function outside of the `with` of your client creation, or if you called `close()` on your client already.
Error invoking function Agents-WebSearchAgent: HTTP transport has already been closed. You may check if you're calling a function outside of the `with` of your client creation, or if you called `close()` on your client already..
Traceback (most recent call last):
  File "c:\Users\dilmurodm\Desktop\Semantic_Kernel\semantic_kernel\venv_py313\Lib\site-packages\semantic_kernel\kernel.py", line 454, in _inner_auto_function_invoke_handler
    result = await context.function.invoke(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<5 lines>...
    )
    ^
  File "c:\Users\dilmurodm\Desktop\Semantic_Kernel\semantic_kernel\venv_py313\Lib\site-packages\semantic_kernel\functions\kernel_function.py", line 259, in invoke
    raise e
  File "c:\Users\dilmurodm\Desktop\Semantic_Kernel\semantic_ker

Assistant > It seems there’s an issue with fetching the latest news directly. Could you provide more details about the Trump tariffs you are interested in, or I can try another way to gather the latest information for you?
